In [1]:
# Import libaries
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

### Data read in

In [2]:
no_dupes = pd.read_csv('../data/imdb_titles_no_dupes.csv')

In [3]:
no_dupes.shape

(2484, 9)

In [4]:
no_dupes.head()

,name,href,years,imdb_description,pg_rating,imdb_genre_tags,imdb_rating,num_votes,img_thumbnail
0,Game of Thrones,/title/tt0944947/,(2011–2019),Nine noble families fight for control over the...,TV-MA,"Action, Adventure, Drama",9.2,2148311,https://m.media-amazon.com/images/M/MV5BYTRiND...
1,Prison Break,/title/tt0455275/,(2005–2017),"Due to a political conspiracy, an innocent man...",TV-14,"Action, Crime, Drama",8.3,548267,https://m.media-amazon.com/images/M/MV5BMTg3NT...
2,Vikings,/title/tt2306299/,(2013–2020),Vikings transports us to the brutal and myster...,TV-MA,"Action, Adventure, Drama",8.5,547494,https://m.media-amazon.com/images/M/MV5BODk4Zj...
3,The Boys,/title/tt1190634/,(2019– ),A group of vigilantes set out to take down cor...,TV-MA,"Action, Comedy, Crime",8.7,542317,https://m.media-amazon.com/images/M/MV5BOTEyND...
4,The Mandalorian,/title/tt8111088/,(2019– ),The travels of a lone bounty hunter in the out...,TV-14,"Action, Adventure, Fantasy",8.7,527088,https://m.media-amazon.com/images/M/MV5BZjRlZD...


In [5]:
base_url = 'https://www.imdb.com'

test_df = no_dupes.head().copy()

### Convert back to list of dictionaries

In [8]:
# Technique reference: https://www.geeksforgeeks.org/create-a-list-from-rows-in-pandas-dataframe/

# Create an empty list
entries_list =[]

# Iterate over each row
for index, rows in test_df.iterrows():
    # Create list for the current row
    my_dict ={'name': rows['name'], 
              'href' : rows['href'], 
              'years' : rows['years'], 
            #   'imdb_description' : rows['imdb_description'], 
              'pg_rating' : rows['pg_rating'], 
              # 'imdb_genre_tags' : rows['imdb_genre_tags'], 
              'imdb_rating' : rows['imdb_rating'],
              'num_votes' : rows['num_votes'],
              'img_thumbnail' : rows['img_thumbnail']
    }
    
    # append the list to the final list
    entries_list.append(my_dict)
  
# Print the list
# print(entries_list)

### Iterate thru list and scrape each URL for descriptions and genre tags

In [9]:
base_url = 'https://www.imdb.com'

for entry in entries_list:
    full_url = base_url + entry['href']
    # print(full_url)

    # Technique reference to avoid 403 error: 
    # https://medium.com/@raiyanquaium/how-to-web-scrape-using-beautiful-soup-in-python-without-running-into-http-error-403-554875e5abed

    req = Request(full_url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, "html.parser")
    
    description_obj = soup.find('p')
    description = description_obj.text

    genre_tags = []
    span_obj = soup.find_all('span', attrs='ipc-chip__text')

    for item in span_obj:
        if item.text != 'Back to top':
            genre_tags.append(item.text)
    
    entry['imdb_description'] = description
    entry['imdb_genre_tags'] = genre_tags


5

In [11]:
entries_list[0:3]

[{'name': 'Game of Thrones',
  'href': '/title/tt0944947/',
  'years': '(2011–2019)',
  'pg_rating': 'TV-MA',
  'imdb_rating': 9.2,
  'num_votes': 2148311,
  'img_thumbnail': 'https://m.media-amazon.com/images/M/MV5BYTRiNDQwYzAtMzVlZS00NTI5LWJjYjUtMzkwNTUzMWMxZTllXkEyXkFqcGdeQXVyNDIzMzcwNjc@._V1_UY98_CR2,0,67,98_AL_.jpg',
  'imdb_description': 'Nine noble families fight for control over the lands of Westeros, while an ancient enemy returns after being dormant for millennia.Nine noble families fight for control over the lands of Westeros, while an ancient enemy returns after being dormant for millennia.Nine noble families fight for control over the lands of Westeros, while an ancient enemy returns after being dormant for millennia.',
  'imdb_genre_tags': ['Action', 'Adventure', 'Drama']},
 {'name': 'Prison Break',
  'href': '/title/tt0455275/',
  'years': '(2005–2017)',
  'pg_rating': 'TV-14',
  'imdb_rating': 8.3,
  'num_votes': 548267,
  'img_thumbnail': 'https://m.media-amazon.com/im

In [101]:
pd.DataFrame(entries_list)

,name,href,verifiation,genre,description,genre_tags
0,Avatar: The Last Airbender,/title/tt0417299/,1. Avatar: The Last Airbender (2005–2008),Action,"In a war-torn world of elemental magic, a youn...","[Animation, Action, Adventure]"
1,Game of Thrones,/title/tt0944947/,2. Game of Thrones (2011–2019),Action,Nine noble families fight for control over the...,"[Action, Adventure, Drama]"
2,Fullmetal Alchemist: Brotherhood,/title/tt1355642/,3. Fullmetal Alchemist: Brotherhood (2009–2010),Action,Two brothers search for a Philosopher's Stone ...,"[Animation, Action, Adventure]"
3,Attack on Titan,/title/tt2560140/,4. Attack on Titan (2013–2023),Action,After his hometown is destroyed and his mother...,"[Animation, Action, Adventure]"
4,Arcane,/title/tt11126994/,5. Arcane (2021– ),Action,Set in Utopian Piltover and the oppressed unde...,"[Animation, Action, Adventure]"


In [46]:
# Alternate method

# # Technique reference: https://stackoverflow.com/questions/36768068/get-meta-tag-content-property-with-beautifulsoup-and-python

# title = soup.find("meta", property="og:title")
# url = soup.find("meta", property="og:url")
# description = soup.find("meta", property="og:description")

# print(title["content"] if title else "No meta title given")
# print(url["content"] if url else "No meta url given")
# print(description["content"] if description else "No meta description given")

Avatar: The Last Airbender (TV Series 2005–2008) - IMDb
https://www.imdb.com/title/tt0417299/
Avatar: The Last Airbender: Created by Michael Dante DiMartino, Bryan Konietzko. With Dee Bradley Baker, Zach Tyler Eisen, Mae Whitman, Jack De Sena. In a war-torn world of elemental magic, a young boy reawakens to undertake a dangerous mystic quest to fulfill his destiny as the Avatar, and bring peace to the world.
